# Jet in cross-flow

Andreas Kitzmüller, Simon Mader

## Ensuring laminar flow

First, we define the kinematic viscosity of air, as used in the simulation, as a global variable:

In [1]:
NU_AIR = 1.5e-5



 find the right dimension to ensure laminar flow. The Reynolds $\text{Re}$ number is given by:

 $\text{Re} = \frac{u \cdot D_h}{\nu_{air}}$, with $D_h = \frac{A}{P}$, with area $A=s$ and perimeter $4 \cdot s$. $\Rightarrow D_h = s \Rightarrow \text{Re} = \frac{u \cdot s}{\nu_{air}}$

In [2]:
Reynolds = lambda s=1.0, u=1.0 : u * s / NU_AIR

In order to find a good geometry, we solve the above formula for u, to be able to set certain side length and velocities and check their Reynolds numbers:

$u = \frac{\text{Re} \cdot \nu_{air}}{s}$

In [3]:
find_velocity = lambda s=1.0, Re=2000.0: Re * NU_AIR / s

To be on the safe side and allow for reasonable cases, we aim for Re less or equal than 2000 for the main pipe and less or equal than 200 for the small duct. In addition, we want to allow for velocities of up to $1\,\mathtt{m/s}$. Therefore, we choose:

In [4]:
print("maximal initial velocity small duct:", find_velocity(s=0.00375, Re=200), "m/s")
print("maximal initial velocity main pipe:", find_velocity(s=0.01125, Re=2000), "m/s")

maximal initial velocity small duct: 0.8 m/s
maximal initial velocity main pipe: 2.666666666666667 m/s


## Simulations

### Case 1: $u_{pipe} = u_{duct} = 0.1 m/s$

<video controls src="videos/U1_0_1_U2_0_1/Z_cut_U_smoothed.mp4" width="1000" />

<video controls src="videos/U1_0_1_U2_0_1/Z_cut_U_mesh.mp4" width="1000" />

### Case 1: $u_{pipe} = 0.1 m/s,  = u_{duct} = 0.6 m/s$

<video controls src="videos/U1_0_1_U2_0_6/Z_cut_U_smoothed.mp4" width="1000" />

<video controls src="videos/U1_0_1_U2_0_6/Z_cut_U_mesh.mp4" width="1000" />

### Case 1: $u_{pipe} = 0.6 m/s,  = u_{duct} = 0.1 m/s$

<video controls src="videos/U1_0_6_U2_0_1/Z_cut_U_smoothed.mp4" width="1000" />

<video controls src="videos/U1_0_6_U2_0_1/Z_cut_U_mesh.mp4" width="1000" />

stuff still to do:

1. track particles
2. more post-processing
3. some more cases ("fountain")